In [ ]:
# This section of the code is for all imports and setup
# Changed from manual file input to automatic batch processing of all BetCen occultation files
# The code now processes all 12 occultation files automatically and generates outputs for each

In [ ]:
# Imports
import os
import glob
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import pandas as pd

# This allows me to access my flash drive that im working off of
os.chdir('/Volumes/Flash Drive')

# ===== AUTOMATIC BATCH PROCESSING =====
# Automatically find all BetCen occultation files matching the pattern BetCen_*_1km.sav
occultation_dir = '/Volumes/Flash Drive/Saturns rings Research/Data/BetCen Occultations/'
occultation_files = sorted(glob.glob(os.path.join(occultation_dir, 'BetCen_*_1km.sav')))

print(f"Found {len(occultation_files)} occultation files to process:")
for i, file in enumerate(occultation_files, 1):
    print(f"  {i}. {os.path.basename(file)}")

# Initial approximate locations of the ringlets (same for all occultations)
saturn_ringlets_approx = {
    'Titan': 77883,      # Colombo Gap
    'Maxwell': 87500,    # Maxwell Gap
    'Bond': 88710,       # Bond Ringlet
    'Huygens': 117800,   # Huygens Gap
    'Dawes': 90210       # Dawes Ringlet
}

print("\n" + "=" * 70)
print("Starting batch processing...")
print("=" * 70)

In [ ]:
# ===== MAIN PROCESSING LOOP =====
# Loop through all occultation files and process each one
# For each file: plot optical depth, find ringlet centers, extract individual data points
# Outputs: PNG screenshots of plots + CSV files with individual measurements

In [ ]:
# Process each occultation file one at a time
for file_idx, file_path in enumerate(occultation_files, 1):
    occultation_file = os.path.basename(file_path)
    
    print(f"\n{'='*70}")
    print(f"Processing file {file_idx}/{len(occultation_files)}: {occultation_file}")
    print(f"{'='*70}\n")
    
    # Reading in the file
    test = sio.readsav(file_path)
    
    # Extract the data structure
    pdsdata = test['pdsdata'][0]
    
    # Get radius and tau to graph radius vs optical depth
    radius = pdsdata['RADIUS']
    tau = pdsdata['TAU']
    
    # ===== STEP 1: Overview plot with all ringlets =====
    plt.figure(figsize=(14, 6))
    plt.plot(radius, tau, 'k-', linewidth=0.5, label='Optical Depth (τ)')
    
    # Add vertical lines for each ringlet
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    for (ringlet_name, ringlet_radius), color in zip(saturn_ringlets_approx.items(), colors):
        plt.axvline(x=ringlet_radius, color=color, linestyle='--', linewidth=1.5, 
                    label=f'{ringlet_name} ({ringlet_radius} km)', alpha=0.3)
    
    # Labels and title
    plt.xlabel('Radius (km)', fontsize=12)
    plt.ylabel('Optical Depth (τ)', fontsize=12)
    plt.title(f'Saturn Ring Optical Depth vs Radius - {occultation_file}', fontsize=14)
    plt.xlim(70000, 120000)
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Save the overview plot
    overview_filename = f'/Volumes/Flash Drive/Saturns rings Research/Screenshot Outputs/{occultation_file.replace(".sav", "")}_overview.png'
    plt.savefig(overview_filename, dpi=300, bbox_inches='tight')
    print(f"Saved overview plot: {overview_filename}")
    plt.show()
    
    print(f"\nProcessing {occultation_file}...")
    print(f"Data radius range: {np.min(radius):.2f} km to {np.max(radius):.2f} km")
    print(f"Number of data points: {len(radius)}\n")
    
    # ===== STEP 2: Zoom into each ringlet ±100 km from approximate center =====
    print("Creating zoom plots...")
    
    # Create 5 separate plots, each zoomed in ±100 km from ringlet center
    fig, axes = plt.subplots(3, 2, figsize=(14, 12))
    axes = axes.flatten()
    
    # Plot each ringlet
    for idx, (ringlet_name, ringlet_radius) in enumerate(sorted(saturn_ringlets_approx.items(), key=lambda x: x[1])):
        ax = axes[idx]
        
        # Define the zoom window: ±100 km from ringlet center
        r_min = ringlet_radius - 100
        r_max = ringlet_radius + 100
        
        # Create mask for this zoom region
        mask = (radius >= r_min) & (radius <= r_max)
        
        # Plot tau vs radius
        ax.plot(radius[mask], tau[mask], 'k-', linewidth=0.8)
        
        # Add vertical line at ringlet center
        ax.axvline(x=ringlet_radius, color='red', linestyle='--', linewidth=1.5, 
                   label=f'{ringlet_name} Ringlet')
        
        # Labels and title
        ax.set_xlabel('Radius (km)', fontsize=10)
        ax.set_ylabel('Optical Depth (τ)', fontsize=10)
        ax.set_title(f'{ringlet_name} Ringlet (centered at {ringlet_radius:,} km)', fontsize=11, fontweight='bold')
        ax.set_xlim(r_min, r_max)
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best', fontsize=9)
        
        # Add text showing the zoom range
        ax.text(0.02, 0.98, f'Range: {r_min:,} - {r_max:,} km', 
                transform=ax.transAxes, fontsize=8, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Remove the 6th empty subplot
    fig.delaxes(axes[5])
    fig.suptitle(f'Ringlet Zoom Views - {occultation_file}', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    
    # Save zoom plot
    zoom_filename = f'/Volumes/Flash Drive/Saturns rings Research/Screenshot Outputs/{occultation_file.replace(".sav", "")}_zoom.png'
    plt.savefig(zoom_filename, dpi=300, bbox_inches='tight')
    print(f"Saved zoom plot: {zoom_filename}")
    plt.show()
    
    # ===== STEP 3: Find ringlet centers and edges using tau threshold crossings =====
    print("\nFinding ringlet centers based on tau threshold crossings:")
    print("=" * 70)
    
    # Initialize dictionaries for this occultation
    saturn_ringlets_refined = saturn_ringlets_approx.copy()
    ringlet_status = {name: 'Approximate center (no analysis)' for name in saturn_ringlets_approx.keys()}
    ringlet_edges = {name: (None, None, None) for name in saturn_ringlets_approx.keys()}
    
    for ringlet_name, ringlet_radius_approx in sorted(saturn_ringlets_approx.items(), key=lambda x: x[1]):
        # Define initial search window: ±100 km from approximate center
        r_min_search = ringlet_radius_approx - 100
        r_max_search = ringlet_radius_approx + 100
        
        # Create mask for search region
        mask_search = (radius >= r_min_search) & (radius <= r_max_search)
        
        # Checking if there is data there
        n_points = np.sum(mask_search)
        print(f"\n{ringlet_name}: Searching {r_min_search:.2f} - {r_max_search:.2f} km, found {n_points} points")
        
        if n_points == 0:
            print(f"  WARNING: No data found in search region! Skipping analysis.")
            saturn_ringlets_refined[ringlet_name] = ringlet_radius_approx
            ringlet_status[ringlet_name] = 'N/A - No Data'
            ringlet_edges[ringlet_name] = (None, None, None)
            continue
        
        # Get data in this region
        tau_in_region = tau[mask_search]
        radius_in_region = radius[mask_search]
        
        # Determine adaptive threshold (20% of max tau)
        max_tau_value = np.max(tau_in_region)
        threshold = 0.2 * max_tau_value 
        print(f"  Max tau: {max_tau_value:.4f}, Using adaptive threshold: {threshold:.4f}")
        
        # Find points where tau crosses the threshold
        above_threshold = tau_in_region >= threshold
        
        if not np.any(above_threshold):
            print(f"  WARNING: No points above tau = {threshold:.4f}! Using max tau approach instead.")
            max_tau_idx = np.argmax(tau_in_region)
            refined_center = radius_in_region[max_tau_idx]
            saturn_ringlets_refined[ringlet_name] = refined_center
            ringlet_status[ringlet_name] = f'Max tau fallback (max τ = {max_tau_value:.4f})'
            ringlet_edges[ringlet_name] = (refined_center, refined_center, threshold)
            print(f"  Refined: {refined_center:.2f} km (max τ = {max_tau_value:.4f})")
            continue
        
        # Find the indices where we cross the threshold
        crossing_indices = np.where(above_threshold)[0]
        left_idx = crossing_indices[0]
        right_idx = crossing_indices[-1]
        
        left_radius = radius_in_region[left_idx]
        right_radius = radius_in_region[right_idx]
        
        # Calculate median (center) between left and right edges
        refined_center = np.median([left_radius, right_radius])
        
        saturn_ringlets_refined[ringlet_name] = refined_center
        ringlet_edges[ringlet_name] = (left_radius, right_radius, threshold)
        ringlet_status[ringlet_name] = f'Adaptive (20% max τ), max τ = {max_tau_value:.4f}'
        
        print(f"  Approx: {ringlet_radius_approx:,} km")
        print(f"  Left edge (τ≥{threshold:.4f}): {left_radius:.2f} km")
        print(f"  Right edge (τ≥{threshold:.4f}): {right_radius:.2f} km")
        print(f"  Refined center (median): {refined_center:.2f} km")
        print(f"  Width: {right_radius - left_radius:.2f} km")
    
    print("\n" + "=" * 70)
    
    # ===== STEP 4: Plot the refined analysis with detected edges =====
    print("\nCreating refined analysis plots...")
    
    # Create 5 separate plots showing refined centers and edges
    fig, axes = plt.subplots(3, 2, figsize=(14, 12))
    axes = axes.flatten()
    
    # Plot each ringlet with refined centers and edges
    for idx, (ringlet_name, ringlet_radius) in enumerate(sorted(saturn_ringlets_refined.items(), key=lambda x: x[1])):
        ax = axes[idx]
        
        # Define the zoom window: ±100 km from ringlet center
        r_min = ringlet_radius - 100
        r_max = ringlet_radius + 100
        
        # Create mask for this zoom region
        mask = (radius >= r_min) & (radius <= r_max)
        
        # Check if we have data to plot
        if np.sum(mask) == 0:
            ax.text(0.5, 0.5, f'N/A - No Data Available\n\n{ringlet_name} Ringlet\nExpected at {ringlet_radius:,.0f} km', 
                    ha='center', va='center', transform=ax.transAxes, fontsize=12,
                    bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
            ax.set_title(f'{ringlet_name} Ringlet - N/A', fontsize=11, fontweight='bold', color='gray')
            ax.set_xlabel('Radius (km)', fontsize=10)
            ax.set_ylabel('Optical Depth (τ)', fontsize=10)
            ax.grid(True, alpha=0.3)
            continue
        
        # Plot tau vs radius
        ax.plot(radius[mask], tau[mask], 'k-', linewidth=0.8)
        
        # Add vertical line at ringlet center and edges
        if ringlet_status[ringlet_name] != 'N/A - No Data':
            ax.axvline(x=ringlet_radius, color='red', linestyle='--', linewidth=1.5, 
                       label=f'{ringlet_name} Center (refined)')
            
            # Get edge information
            left_edge, right_edge, threshold_used = ringlet_edges[ringlet_name]
            
            # Plot the edges and threshold line if available
            if left_edge is not None and right_edge is not None:
                # Plot vertical lines at edges
                ax.axvline(x=left_edge, color='green', linestyle=':', linewidth=1.2, 
                           alpha=0.7, label=f'Left edge (τ≥{threshold_used:.3f})')
                ax.axvline(x=right_edge, color='green', linestyle=':', linewidth=1.2, 
                           alpha=0.7, label=f'Right edge')
                
                # Plot horizontal threshold line
                ax.axhline(y=threshold_used, color='orange', linestyle='-.', linewidth=1, 
                           alpha=0.5, label=f'Threshold τ={threshold_used:.3f}')
            
            # Also show the approximate center for comparison
            approx_center = saturn_ringlets_approx[ringlet_name]
            if r_min <= approx_center <= r_max and abs(approx_center - ringlet_radius) > 1:
                ax.axvline(x=approx_center, color='blue', linestyle=':', linewidth=1, alpha=0.5,
                           label=f'Approximate center')
        else:
            # For N/A status, just show approximate
            ax.axvline(x=ringlet_radius, color='blue', linestyle=':', linewidth=1.5, 
                       label=f'{ringlet_name} Ringlet (approx)')
        
        # Labels and title
        ax.set_xlabel('Radius (km)', fontsize=10)
        ax.set_ylabel('Optical Depth (τ)', fontsize=10)
        
        if ringlet_status[ringlet_name] != 'N/A - No Data':
            ax.set_title(f'{ringlet_name} Ringlet (centered at {ringlet_radius:.2f} km)', 
                         fontsize=11, fontweight='bold')
        else:
            ax.set_title(f'{ringlet_name} Ringlet (approx at {ringlet_radius:.0f} km) - Limited Data', 
                         fontsize=11, fontweight='bold', color='orange')
        
        ax.set_xlim(r_min, r_max)
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best', fontsize=7)
        
        # Add text showing the zoom range and shift/status
        if ringlet_status[ringlet_name] != 'N/A - No Data':
            approx_center = saturn_ringlets_approx[ringlet_name]
            shift = ringlet_radius - approx_center
            left_edge, right_edge, _ = ringlet_edges[ringlet_name]
            
            if left_edge is not None and right_edge is not None:
                width = right_edge - left_edge
                info_text = f'Range: {r_min:.1f} - {r_max:.1f} km\nShift: {shift:.2f} km\nWidth: {width:.2f} km'
            else:
                info_text = f'Range: {r_min:.1f} - {r_max:.1f} km\nShift: {shift:.2f} km'
                
            ax.text(0.02, 0.98, info_text, 
                    transform=ax.transAxes, fontsize=8, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        else:
            ax.text(0.02, 0.98, f'Range: {r_min:.1f} - {r_max:.1f} km\nStatus: N/A', 
                    transform=ax.transAxes, fontsize=8, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))
    
    # Remove the 6th empty subplot
    fig.delaxes(axes[5])
    
    # Add overall title with the filename and method
    fig.suptitle(f'Ringlet Analysis: {occultation_file}\nMethod: Adaptive Threshold (20% max τ)', 
                 fontsize=14, fontweight='bold', y=0.998)
    
    plt.subplots_adjust(hspace=0.4)
    plt.tight_layout()
    
    # Save the refined analysis figure
    analysis_filename = f'/Volumes/Flash Drive/Saturns rings Research/Screenshot Outputs/{occultation_file.replace(".sav", "")}_ringlet_analysis.png'
    plt.savefig(analysis_filename, dpi=300, bbox_inches='tight')
    print(f"Saved analysis plot: {analysis_filename}")
    plt.show()
    
    # Print summary for this occultation
    print(f"\nRinglet Analysis Summary for {occultation_file}:")
    print("=" * 70)
    for ringlet in sorted(saturn_ringlets_refined.keys(), key=lambda x: saturn_ringlets_refined[x]):
        distance = saturn_ringlets_refined[ringlet]
        status = ringlet_status[ringlet]
        approx = saturn_ringlets_approx[ringlet]
        
        if status == 'N/A - No Data':
            print(f"{ringlet:15s}: {distance:.0f} km (approx) - {status}")
        else:
            shift = distance - approx
            left_edge, right_edge, _ = ringlet_edges[ringlet]
            if left_edge is not None and right_edge is not None:
                width = right_edge - left_edge
                print(f"{ringlet:15s}: {distance:.2f} km (shift: {shift:+.2f} km, width: {width:.2f} km) - {status}")
            else:
                print(f"{ringlet:15s}: {distance:.2f} km (shift: {shift:+.2f} km) - {status}")
    
    # ===== STEP 5: Extract all individual data points from each ringlet =====
    print("\n" + "=" * 70)
    print("Extracting individual data points from ringlet edges ± 5 km:")
    print("=" * 70)
    
    # List to store all data points for this occultation
    all_data_points = []
    
    # Get the scalar values once (they're the same for all ringlets)
    b_angle_value = float(pdsdata['B_ANGLE'])
    mu_value = float(pdsdata['MU'])
    
    for ringlet_name, ringlet_center in saturn_ringlets_refined.items():
        print(f"\n{ringlet_name} Ringlet (center at {ringlet_center:.2f} km):")
        
        # Get the edges from the previous analysis
        left_edge, right_edge, threshold = ringlet_edges[ringlet_name]
        
        if left_edge is None or right_edge is None:
            print(f"  WARNING: No edge data available! Skipping.")
            continue
        
        # Extend 5 km beyond each edge
        r_min = left_edge - 5
        r_max = right_edge + 5
        
        ringlet_width = right_edge - left_edge
        extended_width = r_max - r_min
        
        print(f"  Ringlet edges: {left_edge:.2f} km to {right_edge:.2f} km (width: {ringlet_width:.2f} km)")
        print(f"  Extended range: {r_min:.2f} km to {r_max:.2f} km (total: {extended_width:.2f} km)")
        
        # Create mask for this window
        mask = (radius >= r_min) & (radius <= r_max)
        n_points = np.sum(mask)
        
        print(f"  Number of individual data points: {n_points}")
        print(f"  Expected ~{extended_width:.0f} points (at 1km resolution)")
        
        if n_points == 0:
            print(f"  WARNING: No data in this window!")
            continue
        
        # Extract all individual data points (no averaging!)
        radius_values = radius[mask]
        et_values = pdsdata['ET'][mask]
        moments = pdsdata['MOMENTS200'][mask]  # Shape will be (n_points, 4)
        int_area2_values = pdsdata['INT_AREA2'][mask]
        i0_values = pdsdata['I0'][mask]
        phi_values = pdsdata['PHI'][mask]
        tauplus_values = pdsdata['TAUPLUS'][mask]
        tauminus_values = pdsdata['TAUMINUS'][mask]
        lon_values = pdsdata['LON'][mask]
        tau_values = tau[mask]
        
        # Create a record for each individual data point
        for i in range(n_points):
            data_point = {
                'Occultation': occultation_file.replace('.sav', ''),
                'Ringlet': ringlet_name,
                'Ringlet_Center': ringlet_center,
                'Left_Edge': left_edge,
                'Right_Edge': right_edge,
                'Radius': radius_values[i],
                'Tau': tau_values[i],
                'ET': et_values[i],
                'MOMENTS200_0': moments[i, 0],
                'MOMENTS200_1': moments[i, 1],
                'MOMENTS200_2': moments[i, 2],
                'MOMENTS200_3': moments[i, 3],
                'INT_AREA2': int_area2_values[i],
                'I0': i0_values[i],
                'PHI': phi_values[i],
                'B_ANGLE': b_angle_value,
                'TAUPLUS': tauplus_values[i],
                'TAUMINUS': tauminus_values[i],
                'LON': lon_values[i]
            }
            all_data_points.append(data_point)
    
    print("\n" + "=" * 70)
    
    # Convert to DataFrame - now each row is an individual measurement point
    df = pd.DataFrame(all_data_points)
    
    # Save to CSV for this occultation
    output_csv = f'/Volumes/Flash Drive/Saturns rings Research/Data From Center of Ringlets CSV files/{occultation_file.replace(".sav", "")}_ringlet_individual_points.csv'
    df.to_csv(output_csv, index=False)
    print(f"\nData saved to: {output_csv}")
    print(f"Total data points saved: {len(df)}")
    print(f"\nPreview of saved data (first 10 rows):")
    print(df.head(10))
    print(f"\nData points per ringlet:")
    print(df.groupby('Ringlet').size())
    
    print(f"\n{'='*70}")
    print(f"Completed processing: {occultation_file}")
    print(f"{'='*70}\n")

In [ ]:
# This cell is no longer needed - all processing is in the main loop above

In [ ]:
# This cell is no longer needed - all processing is in the main loop above

In [ ]:
# This cell is no longer needed - all processing is in the main loop above

In [ ]:
# This cell is deprecated - kept for documentation purposes only
# We now use the threshold-based approach below instead of max tau

In [ ]:
# This cell is no longer needed - all processing is in the main loop above

In [ ]:
# This cell is no longer needed - all processing is in the main loop above

In [ ]:
# This cell is no longer needed - all processing is in the main loop above

In [ ]:
# ===== BATCH PROCESSING COMPLETE =====
print("\n" + "="*70)
print("ALL OCCULTATION FILES PROCESSED SUCCESSFULLY!")
print("="*70)
print(f"\nTotal files processed: {len(occultation_files)}")
print("\nOutputs generated for each occultation:")
print("  1. Overview plot (PNG)")
print("  2. Zoom plot (PNG)")
print("  3. Ringlet analysis plot with edges (PNG)")
print("  4. Individual data points CSV file")
print("\nAll files saved to:")
print("  - Screenshots: /Volumes/Flash Drive/Saturns rings Research/Screenshot Outputs/")
print("  - CSV files: /Volumes/Flash Drive/Saturns rings Research/Data From Center of Ringlets CSV files/")
print("="*70)

In [ ]:
# End of notebook